### You must implement a test executable named test.py that imports your framework and

• Generates a training and a test set of 1,000 points sampled uniformly in [0,1]^2, each with a
label 0 if outside the disk centered at (0.5, 0.5) of radius 1/rtsquare(2π), and 1 insid

• builds a network with two input units, two output units, three hidden layers of 25 units 

• trains it with MSE, logging the loss

• computes and prints the final train and the test errors.

You are free to come with any new ideas you want, and grading will reward originality. The suggested simple structure is to define a class

#### class Module(object):
    def forward(self, *input): 
        raise NotImplementedError
    def backward(self, *gradwrtoutput): 
        raise NotImplementedError
    def param(self): 
        return []

and to implement several modules and losses that inherit from it.

Each such module may have tensor parameters, in which case it should also have for each a similarly sized tensor gradient to accumulate the gradient during the back-pass, and

• forward should get for input, and returns, a tensor or a tuple of tensors.

• backward should get as input a tensor or a tuple of tensors containing the gradient of the loss with respect to the module’s output, accumulate the gradient w.r.t. the parameters, and return a tensor or a tuple of tensors containing the gradient of the loss w.r.t. the module’s input.

• param should return a list of pairs, each composed of a parameter tensor, and a gradient tensor of same size. 

This list should be empty for parameterless modules (e.g. ReLU).

Some modules may requires additional methods, and some modules may keep track of information from the forward pass to be used in the backward.

You should implement at least the modules Linear (fully connected layer), ReLU, Tanh, Sequential to combine several modules in basic sequential structure, and LossMSE to compute the MSE loss.

In [1]:
import math
import torch
torch.set_grad_enabled(False)
from torch import empty
import numpy as np


In [2]:
# Create a training set: 1000 points in [0,1]*[0,1]
x_train = torch.rand(1000)
y_train = torch.rand(1000)

# Create a testing set: 1000 points in [0,1]*[0,1]
x_test = torch.rand(1000)
y_test = torch.rand(1000)

# Labelization of the training and testing set: Label '1' if the point is inside the circle, '0' if outside
cercle_center=torch.tensor([[0.5],[0.5]])
label_0=[]
label_1=[]
a = torch.square(x_train-cercle_center[0])
b = torch.square(y_train-cercle_center[1])
R = 1/math.sqrt(2*math.pi)


for i in range(len(x_train)):
    if a[i] + b[i] <= R*R:
        label_1.append([x_train[i],y_train[i]])
    else:
        label_0.append([x_train[i],y_train[i]])

print(len(label_0),len(label_1))   

519 481


In [3]:
# MSE loss function
#y_prediction= tanh()
#y_target= 

def MSE_loss(y_pred,y_true):
    loss= (y_pred-y_true).pow(2).mean() 
    return loss

#loss= MSE_loss(y_prediction, y_target)

#print ("MSE error is: ", loss )

In [4]:
class Connection:
    def __init__(self, connectedNeuron):
        self.connectedNeuron = connectedNeuron
        ##self.weight = np.random.normal()
        self.weight = torch.rand(1)
        self.dWeight = torch.tensor([0])
        


In [5]:
class Neuron:
    eta = torch.tensor([0.001])
    alpha = torch.tensor([0.01])

    def __init__(self, layer):
        self.dendrons = []
        self.error = torch.tensor([0])
        self.gradient = torch.tensor([0])
        self.output = torch.tensor([0])
        if layer is None:
            pass
        else:
            for neuron in layer:
                con = Connection(neuron)
                self.dendrons.append(con)

    def addError(self, err):
        self.error = self.error + err
        
#----------------------------------------------
# Not necessary     
###    def sigmoid(self, x):
###        return 1 / (1 + math.exp(-x * 1.0))

###    def dSigmoid(self, x):
###        return (math.exp(-x * 1.0)) / (1 + math.exp(-x * 1.0))**2
        ##return x*(1-x)
#----------------------------------------------
    
    def tanh(self, x):
        return math.tanh(x)

    def dtanh(self, x):
        return 1 - math.tanh(x)**2

#----------------------------------------------
    
    def Relu(self, x):
        return max(0,x)

    def dRelu(self, x):
        if x<0:
            return 0
        else:
            return 1

#----------------------------------------------

    def setError(self, err):
        self.error = err

    def setOutput(self, output):
        self.output = output

    def getOutput(self):
        return self.output
                
    def backPropagate(self):
        ##self.gradient = self.error * self.dSigmoid(self.output);
        self.gradient = self.error * self.dtanh(self.output);
        for dendron in self.dendrons:
            dendron.dWeight = Neuron.eta * (dendron.connectedNeuron.output * self.gradient) + self.alpha * dendron.dWeight;
            dendron.weight = dendron.weight + dendron.dWeight;
            dendron.connectedNeuron.addError(dendron.weight * self.gradient);
        self.error = 0;
        
    def feedForward(self):
        sumOutput = torch.tensor([0])
        if len(self.dendrons) == 0:    
            return
        for dendron in self.dendrons:
            #print(len(self.dendrons))
            sumOutput = sumOutput +  dendron.connectedNeuron.getOutput() * dendron.weight;
        ##self.output = self.sigmoid(sumOutput)
        self.output = self.tanh(sumOutput)

In [6]:
class Network:
    def __init__(self, topology):
        self.layers = []
        for numNeuron in topology:
            layer = []
            for i in range(numNeuron):
                if (len(self.layers) == 0):
                    layer.append(Neuron(None))
                else:
                    layer.append(Neuron(self.layers[-1]))
            layer.append(Neuron(None))
            layer[-1].setOutput(1)
            self.layers.append(layer)

    def setInput(self, inputs):
        for i in range(len(inputs)):
            self.layers[0][i].setOutput(inputs[i])

    def feedForward(self):
        for layer in self.layers[1:]:
            for neuron in layer:
                neuron.feedForward();

    def backPropagate(self, target):
        for i in range(len(target)):
            self.layers[-1][i].setError(target[i] - self.layers[-1][i].getOutput())
        for layer in self.layers[::-1]:
            for neuron in layer:
                neuron.backPropagate()

    def getError(self, target):
        err = 0
        for i in range(len(target)):
            e = (target[i] - self.layers[-1][i].getOutput())
            err = err + e ** 2
        err = err / len(target)
        err = math.sqrt(err)
        return err

    def getResults(self):
        output = []
        for neuron in self.layers[-1]:
            output.append(neuron.getOutput())
        output.pop()
        return output

    def getThResults(self):
        output = []
        for neuron in self.layers[-1]:
            o = neuron.getOutput()
            if (o >= 0.5):
                o = 1
            else:
                o = 0
            output.append(o)
        output.pop()
        return output


In [15]:
def main():
    
    topology = []
    
    topology.append(2)     # Input layer
    topology.append(25)    # Hidden layer 1
    topology.append(25)    # Hidden layer 2
    topology.append(25)    # Hidden layer 3
    topology.append(1)     # Output layer

    net = Network(topology)
    
    Neuron.eta = torch.tensor([0.09])             # Learning rate of the neurons
    Neuron.alpha = torch.tensor([0.015])          # momentum factor of the neurons
    
    
    while True:           

        err = 0
        ##inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]
        ##outputs = [[0], [0], [1], [0]]
        inputs = [[0, 0]]
        outputs = [[0]]
        print(len(inputs))
        print(len(outputs))
        
        for i in range(len(inputs)):
            net.setInput(inputs[i])
            net.feedForward()
            net.backPropagate(outputs[i])
            err = err + net.getError(outputs[i])
            
        print ("error: ", err)
        
        if err <= 3:    ########### INITIALEMENT A 0.01 mais mis à 3 pour accelerer les tests
            break

            
    while True:
        
        a = input("type 1st input :")
        b = input("type 2nd input :")
        net.setInput([a, b])
        net.feedForward()
        print ('net.getThResults()', net.getThResults())


if __name__ == '__main__':
    
    main()

1
1
error:  0.9999999999998671
type 1st input :[0, 0]
type 2nd input :[0, 1]


TypeError: only integer tensors of a single element can be converted to an index